In [1]:
import numpy as np
import pickle
import pandas as pd

In [2]:
original_auis_filename='/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/INTERSECT_AUI2ID.PICKLE'
new_auis_filename='/data/Bodenreider_UMLS_DL/Interns/Vishesh/eval_umls/UNIQUE_AUI2ID.PICKLE'

original_auis = pickle.load(open(original_auis_filename, 'rb'))
new_auis = pickle.load(open(new_auis_filename, 'rb'))

In [3]:
original_auis = list(original_auis.keys())
new_auis = list(new_auis.keys())

### Original Vishesh Split

In [6]:
pickle.dump(original_auis,open('../data/vishesh_dataset_original_auis.p','wb'))
pickle.dump(new_auis,open('../data/vishesh_dataset_new_auis.p','wb'))

### Original Vishesh Split Subset

In [7]:
pickle.dump(original_auis[:1000],open('../data/vishesh_dataset_subset_original_auis.p','wb'))
pickle.dump(new_auis[:1000],open('../data/vishesh_dataset_subset_new_auis.p','wb'))

### Subset of 2020AA to obtain within 2020AA training pairs

In [5]:
from UMLS import UMLS

In [6]:
umls = UMLS()

Loading Raw MRCONSO Lines


15229292it [00:52, 292055.63it/s]                                                                                                      


Creating mappings between concept IDs for easy access.


 76%|███████████████████████████████████████████████████████████████▊                    | 11577955/15229292 [02:17<06:31, 9321.72it/s]

('A24622297', 'C3277832', 'DNAJB6, PRO96ARG', '', 'OMIM', 'P', 'PF', 'ENG', 'L10455876')


 89%|██████████████████████████████████████████████████████████████████████████▍         | 13499973/15229292 [02:41<03:32, 8157.16it/s]

('A28942758', 'C4310709', 'EPILEPSY, FAMILIAL FOCAL, WITH VARIABLE FOCI 2', '', 'MTH', 'P', 'PF', 'ENG', 'L13793594')


100%|███████████████████████████████████████████████████████████████████████████████████| 15229292/15229292 [02:53<00:00, 87928.17it/s]


In [7]:
original_auis

['A26634265',
 'A26634266',
 'A28315139',
 'A28572604',
 'A0016515',
 'A1317708',
 'A26674543',
 'A1317687',
 'A26661070',
 'A1317707',
 'A18399186',
 'A23513030',
 'A26596366',
 'A26607235',
 'A26604498',
 'A0528280',
 'A26631676',
 'A0100864',
 'A0016535',
 'A0016536',
 'A27769867',
 'A18243499',
 'A23497527',
 'A0016529',
 'A1945338',
 'A1957372',
 'A26663792',
 'A27781005',
 'A0054980',
 'A26596367',
 'A0064292',
 'A26612560',
 'A0032517',
 'A0054992',
 'A0526601',
 'A27766654',
 'A27780666',
 'A26606894',
 'A1317486',
 'A0016576',
 'A0016478',
 'A26626090',
 'A0061802',
 'A26663791',
 'A0526400',
 'A0526764',
 'A0526362',
 'A26647665',
 'A0528180',
 'A26666443',
 'A26658464',
 'A0527273',
 'A0016587',
 'A0088302',
 'A7560642',
 'A26647743',
 'A18626755',
 'A26645065',
 'A0579409',
 'A23862581',
 'A7565702',
 'A3230610',
 'A1199224',
 'A3561643',
 'A18552522',
 'A0016589',
 'A0016488',
 'A26601886',
 'A0016489',
 'A26666507',
 'A0088291',
 'A26661149',
 'A0089769',
 'A26601885',
 '

In [19]:
sgs = []
num_syms = []

for aui in original_auis:
    sgs.append(umls.aui2sg[aui])
    num_syms.append(len(umls.cui2auis[umls.aui2cui[aui]]))

In [23]:
df = pd.DataFrame(original_auis,columns=['aui'])
df['sg'] = sgs
df['num_syns'] = num_syms
df['no_syms'] = [n == 1 for n in df['num_syns']]

In [22]:
df.num_syns

0           2
1           2
2          16
3          16
4          16
           ..
8521215     2
8521216     2
8521217     1
8521218     1
8521219     1
Name: num_syns, Length: 8521220, dtype: int64

In [24]:
display(df.groupby('no_syms').count())
display(df.groupby('no_syms').count()/len(df))

,aui,sg,num_syns
no_syms,,,
False,6147794,6147794,6147794
True,2373426,2373426,2373426


,aui,sg,num_syns
no_syms,,,
False,0.721469,0.721469,0.721469
True,0.278531,0.278531,0.278531


In [37]:
len(df)

8521220

In [38]:
#Stratified Semantic Group Sampling

test_perc = 0.05

train = []
test = []

for i, g in df.groupby('sg'):
    perm_g = g.sample(len(g), random_state=np.random.RandomState(42))

    train.append(perm_g[:len(g) - int(len(g) * (test_perc))])
    test.append(perm_g[len(g) - int(len(g) * test_perc):])

In [39]:
train_df = pd.concat(train)
test_df = pd.concat(test)

In [42]:
print(len(train_df))
display(train_df.groupby('no_syms').count())
display(train_df.groupby('no_syms').count()/len(train_df))

8095168


,aui,sg,num_syns
no_syms,,,
False,5840412,5840412,5840412
True,2254756,2254756,2254756


,aui,sg,num_syns
no_syms,,,
False,0.721469,0.721469,0.721469
True,0.278531,0.278531,0.278531


In [43]:
print(len(test_df))
display(test_df.groupby('no_syms').count())
display(test_df.groupby('no_syms').count()/len(test_df))

426052


,aui,sg,num_syns
no_syms,,,
False,307382,307382,307382
True,118670,118670,118670


,aui,sg,num_syns
no_syms,,,
False,0.721466,0.721466,0.721466
True,0.278534,0.278534,0.278534


In [44]:
original_auis = train_df.aui.values
new_auis = test_df.aui.values

In [45]:
pickle.dump(original_auis,open('../data/2020AA_split.sem_group_stratified.original_auis.p','wb'))
pickle.dump(new_auis,open('../data/2020AA_split.sem_group_stratified.new_auis.p','wb'))